# ❤️ Myocardial Infarction (MI) Detection from 12-Lead ECG (PTB-XL Dataset)

This notebook implements an automated framework for classifying ECG signals using **classical Machine Learning** and **Deep Learning** techniques.

We compare two distinct approaches for **Binary Classification** (Normal vs. Abnormal):
1.  **Feature-Based Approach (ML):** Extracting morphological features using Median Beats and PCA, classified via Logistic Regression and Random Forest.
2.  **End-to-End Approach (DL):** Using Raw Signal data fed directly into a 1D Convolutional Neural Network (CNN).

### 🔹 Key Methodologies
* **Dataset:** PTB-XL (500 Hz), utilizing **all 12 Leads**.
* **Preprocessing:** Bandpass filtering (0.5-50 Hz) & R-Peak detection on Lead II.
* **Feature Engineering (Phase 2):**
    * **Median Beat Construction:** Robust "Territory Logic" (dynamic windowing) to handle Tachycardia/Bradycardia.
    * **Dimensionality Reduction:** Universal PCA to extract **60 standardized features** (5 components per lead).
* **Models:**
    * **Logistic Regression:** Baseline linear classifier on PCA features.
    * **Random Forest:** Non-linear ensemble classifier on PCA features.
    * **1D-CNN:** Deep learning model trained on **Raw 12-lead Time-Series** (5000 samples).

### 🔹 Goal
To evaluate if engineered morphological features (PCA) can compete with the raw signal learning capabilities of Convolutional Neural Networks for detecting cardiac abnormalities.

## Google Colab Requirements

In [ ]:
%pip install wfdb --no-deps

## Import and Configuration

In [ ]:
# --- Standard Library ---
import os
import json
import random
from pathlib import Path

# --- Data Manipulation & Math ---
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib

# --- Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Signal Processing & Domain Specific ---
import wfdb
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis

# --- Scikit-Learn (Machine Learning) ---
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, ConfusionMatrixDisplay
)

# --- TensorFlow / Keras (Deep Learning) ---
import tensorflow as tf
from keras import mixed_precision
from keras import Sequential, backend as K
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.layers import (
    Input, Conv1D, MaxPooling1D, Dense, Dropout, SpatialDropout1D,
    Activation,BatchNormalization, Add, GlobalAveragePooling1D, Concatenate,
    Lambda, Reshape, UpSampling1D, Conv1DTranspose, Layer, Cropping1D
)

# --- Configuration & Constants ---
INPUT_DIR = "/kaggle/input/"
OUTPUT_DIR = "/kaggle/working/"
DATASET_PATH = os.path.join(INPUT_DIR, "ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1")
META_FILE_PTB = os.path.join(DATASET_PATH, "ptbxl_database.csv")
META_FILE_SCP = os.path.join(DATASET_PATH, "scp_statements.csv")

SEED = 42
SAMPLE_RATE = 500
TEST_SIZE = 0.2
MAX_RECORDS = 3000

CNN_EPOCHS = 50
BATCH_SIZE = 32

# Set Seeds
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Helper Functions

In [ ]:
def safe_eval_scp(scp_code):
    try: return eval(scp_code) if isinstance(scp_code, str) else scp_code
    except: return {}

def ensure_len(sig, expected):
    # sig shape: (time, leads) or (time,)
    length = len(sig)
    if length == expected:
        return sig

    if length > expected:
        return sig[:expected]

    # Padding logic for 2D (12 leads) or 1D (single lead)
    diff = expected - length
    if sig.ndim == 1:
        pad = np.zeros(diff)
        return np.concatenate([sig, pad])
    else:
        # Assume shape (time, leads), pad along axis 0 (time)
        leads = sig.shape[1]
        pad = np.zeros((diff, leads))
        return np.concatenate([sig, pad], axis=0)

def normalize_per_record(X):
    # X shape: (n_samples, signal_len, n_leads)
    # Mean and Std along the signal_len dimension (axis 1)
    m = X.mean(axis=1, keepdims=True)
    s = X.std(axis=1, keepdims=True) + 1e-8
    return (X - m) / s

def print_result(res):
    print(f"\nModel: {res['model']}")
    print(f"Accuracy: {res['accuracy']:.2%}")
    print(f"Precision: {res['precision']:.2%}")
    print(f"Recall: {res['recall']:.2%}")
    print(f"F1 Score: {res['f1 score']:.2%}")

## Load Metadata

In [ ]:
def load_metadata(max_records=MAX_RECORDS):
    # Load raw dataframes
    ptb_df = pd.read_csv(META_FILE_PTB)
    scp_df = pd.read_csv(META_FILE_SCP, index_col=0)

    # Convert scp_codes from string to dict
    ptb_df["scp_codes"] = ptb_df["scp_codes"].apply(safe_eval_scp)

    # Drop records without a filename
    ptb_df = ptb_df[ptb_df["filename_hr"].notna()].reset_index(drop=True)

    # --- Step 1: Build the Dictionary Map (Code -> Superclass) ---
    # We only keep codes that have a valid diagnostic_class (MI, STTC, CD, HYP, NORM)
    scp_df = scp_df[scp_df["diagnostic_class"].notna()]
    code_map = scp_df["diagnostic_class"].to_dict()

    # --- Step 2: Define the Labeling Function with Hierarchy ---
    def get_label(scp_dict):
        # Get all superclasses present for this patient
        classes = set()
        for code in scp_dict.keys():
            if code in code_map:
                classes.add(code_map[code])

        # Check Priority: MI > STTC > CD > HYP > NORM
        if "MI" in classes: return 1      # MI
        if "STTC" in classes: return 2    # STTC
        if "CD" in classes: return 3      # CD
        if "HYP" in classes: return 4     # HYP
        if "NORM" in classes: return 0    # NORM

        return -1 # Unknown/Other (will be dropped)

    # --- Step 3: Apply Labels ---
    ptb_df["label_multiclass"] = ptb_df["scp_codes"].apply(get_label)

    # Drop "Other/Unknown" (-1)
    ptb_df = ptb_df[ptb_df["label_multiclass"] != -1].reset_index(drop=True)

    # Create Binary Label: 0=NORM, 1=Abnormal (MI, STTC, CD, HYP)
    ptb_df["label_binary"] = ptb_df["label_multiclass"].apply(lambda x: 0 if x == 0 else 1)

    # --- Step 4: Stratified Sampling (Optional) ---
    if max_records is not None and max_records < len(ptb_df):
        from sklearn.model_selection import train_test_split
        # Stratify by multiclass label to keep distribution balanced
        ptb_df, _ = train_test_split(
            ptb_df,
            train_size=max_records,
            stratify=ptb_df["label_multiclass"],
            random_state=SEED,
        )
        ptb_df = ptb_df.reset_index(drop=True)

    return ptb_df

ptb_df = load_metadata()
print("Total records used:", len(ptb_df))
print("\nMulticlass Counts (0=NORM, 1=MI, 2=STTC, 3=CD, 4=HYP):")
print(ptb_df["label_multiclass"].value_counts().sort_index())
print("\nBinary Counts (0=Normal, 1=Abnormal):")
print(ptb_df["label_binary"].value_counts())
ptb_df[["ecg_id", "scp_codes", "label_multiclass", "label_binary"]]

## Load One Signal and Smoke Test

In [ ]:
def load_signal_single(path_record):
    """
    Loads all 12 leads of the ECG signal.
    Returns: numpy array of shape (time_steps, 12)
    """
    try:
        recp = os.path.join(DATASET_PATH, path_record)
        # wfdb.rdsamp returns (signal, fields). We only need signal.
        sig, fields = wfdb.rdsamp(recp)
        return sig.astype(np.float32)
    except:
        return None

def smoke_test(df, count=1):
    print(f"Showing first {count} records (12-Lead view)...")

    lead_names = ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

    for row in df.itertuples():
        sig = load_signal_single(row.filename_hr)
        if sig is None:
            continue

        # Ensure length (now handles 12 leads)
        sig = ensure_len(sig, SAMPLE_RATE * 10)

        # Plot all 12 leads stacked
        fig, axes = plt.subplots(12, 1, figsize=(10, 15), sharex=True)

        for i in range(12):
            ax = axes[i]
            ax.plot(sig[:, i], color='black', linewidth=0.8)
            ax.set_ylabel(lead_names[i], rotation=0, labelpad=20, fontsize=10, fontweight='bold')
            ax.grid(True, linestyle='--', alpha=0.5)
            if i < 11:
                ax.tick_params(labelbottom=False)

        plt.suptitle(f"Record: {row.filename_hr}\nBinary: {row.label_binary} | Multi: {row.label_multiclass}", y=1.02)
        plt.xlabel("Time (samples)")
        plt.tight_layout()
        plt.show()

        count -= 1
        if count == 0:
            return

smoke_test(df=ptb_df)

## Build Raw Dataset

In [ ]:
def build_raw_dataset(df):
    X, y_bin, y_multi, rec_ids = [], [], [], []

    for r in tqdm(df.itertuples(), desc="Loading ECG signals"):
        # Load the full 12-lead signal
        sig = load_signal_single(r.filename_hr)
        if sig is None:
            continue

        # Ensure length is exactly 5000 samples (handles 12 leads automatically)
        sig = ensure_len(sig, SAMPLE_RATE * 10)

        X.append(sig)
        y_bin.append(r.label_binary)
        y_multi.append(r.label_multiclass)
        rec_ids.append(r.filename_hr)

    # Stack to create 3D Array: (Samples, Time, Leads)
    X = np.stack(X)
    y_bin = np.array(y_bin)
    y_multi = np.array(y_multi)

    return X, y_bin, y_multi, rec_ids

# --- Execute the Build ---
X, y_bin, y_multi, rec_ids = build_raw_dataset(ptb_df)

# --- Normalize ---
X = normalize_per_record(X)

print("\nDataset Built!")
print("X shape:", X.shape)
print("y_bin shape:", y_bin.shape)
print("y_multi shape:", y_multi.shape)

## Feature Extraction

In [ ]:
WINDOW_LEN = 300
PRE_R = 100
POST_R = 200
SAFETY_MARGIN = 10

N_PCA_COMPONENTS = 5


def get_median_beats(sig):
    """
    Extracts the median beat from a 12-lead signal.
    Input: sig (5000, 12)
    Output: median_beat (12, 300) -> Transposed for PCA consistency
    """
    # 1. Detect Peaks on Lead II (Index 1)
    peaks, _ = find_peaks(sig[:, 1], distance=150, height=0.1)

    # If no peaks, return zeros (12 leads x 300 samples)
    if len(peaks) == 0:
        return np.zeros((12, WINDOW_LEN), dtype=np.float32)

    beats = []

    for i, r_curr in enumerate(peaks):
        # --- A. Define Ideal Territory ---
        ideal_start = r_curr - PRE_R
        ideal_end = r_curr + POST_R

        # --- B. Define Safety Limits ---
        if i > 0:
            limit_start = peaks[i-1] + POST_R + SAFETY_MARGIN
        else:
            limit_start = 0

        if i < len(peaks) - 1:
            limit_end = peaks[i+1] - PRE_R - SAFETY_MARGIN
        else:
            limit_end = sig.shape[0]

        # --- C. Truncate ---
        actual_start = max(ideal_start, limit_start)
        actual_end = min(ideal_end, limit_end)

        actual_start = max(0, actual_start)
        actual_end = min(sig.shape[0], actual_end)

        # --- D. The Canvas Method ---
        canvas = np.zeros((WINDOW_LEN, 12), dtype=np.float32)
        canvas_start_idx = PRE_R - (r_curr - actual_start)

        chunk = sig[actual_start:actual_end]
        chunk_len = len(chunk)

        if chunk_len == 0 or canvas_start_idx < 0 or (canvas_start_idx + chunk_len) > WINDOW_LEN:
            continue

        canvas[canvas_start_idx : canvas_start_idx + chunk_len] = chunk
        beats.append(canvas)

    # --- E. Compute Median ---
    if len(beats) == 0:
        return np.zeros((12, WINDOW_LEN), dtype=np.float32)

    beats = np.stack(beats) # Shape: (N_beats, 300, 12)
    median_beat = np.median(beats, axis=0) # Shape: (300, 12)

    # Transpose to return (12, 300)
    return median_beat.T

def train_pca_model(X):
    """
    Trains PCA on training set.
    Input: X (N, 5000, 12)
    Output: Trained PCA object
    """
    print("Preparing data for PCA training...")
    all_beats = []

    for x in tqdm(X, desc="Extracting Training Beats"):
        mb = get_median_beats(x) # Returns (12, 300)
        all_beats.append(mb)     # List of (12, 300) arrays

    # Stack: (N_patients * 12, 300)
    X_for_pca = np.vstack(all_beats)

    print(f"Training PCA on {X_for_pca.shape[0]} individual lead waveforms...")
    pca = PCA(n_components=N_PCA_COMPONENTS)
    pca.fit(X_for_pca)

    print(f"PCA Variance Explained: {np.sum(pca.explained_variance_ratio_):.2%}")
    return pca

def extract_pca_features(X, pca_model):
    """
    Transforms signals into features.
    Input: X (N, 5000, 12)
    Output: features (N, 60)
    """
    features = []

    for x in tqdm(X, desc="Transforming Features"):
        mb = get_median_beats(x) # (12, 300)

        # Project: Input (12, 300) -> Output (12, 5)
        coeffs = pca_model.transform(mb)

        # Flatten: 12 * 5 = 60 features
        features.append(coeffs.flatten())

    return np.array(features)

## Train Test Split and Features Extraction

In [ ]:
# 1. Stratified Split (Raw Data)
X_train, X_test, y_train_bin, y_test_bin, y_train_multi, y_test_multi = train_test_split(
    X,
    y_bin,
    y_multi,
    test_size=TEST_SIZE,
    stratify=y_multi,
    random_state=SEED
)

print(f"Split Sizes: Train={len(X_train)}, Test={len(X_test)}")

# 2. Train PCA (On Training Set Only)
pca = train_pca_model(X_train)

# 3. Extract Features
print("\nExtracting features for Training Set...")
Xf_train = extract_pca_features(X_train, pca)

print("\nExtracting features for Test Set...")
Xf_test = extract_pca_features(X_test, pca)

# 4. Scale Features
scaler = StandardScaler()
Xf_train = scaler.fit_transform(Xf_train)
Xf_test = scaler.transform(Xf_test)

## Features Display

In [ ]:
# --- Flexible Feature Display Function ---
def show_feature_table(features, lead_idx=None):
    """
    Creates a DataFrame for the features and displays the head.
    lead_idx: Integer (0-11) to filter for a specific lead.
              If None, displays all 60 features.
    """
    # 1. Generate Column Names
    lead_names = ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    col_names = [f"{lead}_C{i+1}" for lead in lead_names for i in range(N_PCA_COMPONENTS)]

    # 2. Create Full DataFrame
    df = pd.DataFrame(features, columns=col_names)

    # 3. Filter or Show All
    if lead_idx is not None:
        if 0 <= lead_idx < 12:
            target_lead = lead_names[lead_idx]
            # Select columns containing the lead name (e.g., "II_C1", "II_C2"...)
            # We use specific slicing math: start = idx*5, end = start+5
            start_col = lead_idx * N_PCA_COMPONENTS
            end_col = start_col + N_PCA_COMPONENTS

            print(f"\n--- Features for Lead {target_lead} (Index {lead_idx}) ---")
            display(df.iloc[:, start_col:end_col].head())
        else:
            print(f"Error: Lead index must be 0-11.")
    else:
        print("\n--- All 60 Features (12 Leads x 5 Components) ---")
        display(df.head())

# --- Usage Examples ---
# Example 1: Show Lead II (Index 1) - Often the most important lead
show_feature_table(Xf_train, lead_idx=1)

# Example 2: Show All Features
# show_feature_table(Xf_train)

## Universal Evaluator

In [ ]:
def evaluate_model(model_name, y_true, y_pred):
    """
    Calculates metrics and displays confusion matrix.
    Returns a dictionary of metrics.
    """
    print(f"\n{'='*40}")
    print(f"EVALUATION: {model_name}")
    print(f"{'='*40}")

    # 1. Calculate Metrics
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # 2. Print Report
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print("-" * 20)
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['Normal', 'Abnormal']))

    # 3. Plot Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal', 'Abnormal'])

    fig, ax = plt.subplots(figsize=(5, 5))
    disp.plot(cmap='Blues', ax=ax, colorbar=False)
    ax.set_title(f"{model_name} Confusion Matrix")
    plt.show()

    # 4. Return Object
    return {
        "model": model_name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1 score": f1
    }


## Machine Learning Models Training

In [ ]:
# Train Logistic Regression Model
lr_model = LogisticRegression(max_iter=1000, random_state=SEED)
print("Training Logistic Regression on PCA Features...")
lr_model.fit(Xf_train, y_train_bin)

# Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=200, random_state=SEED)
print("Training Random Forest on PCA Features...")
rf_model.fit(Xf_train, y_train_bin)

## Machine Learning Models Evaluation

In [ ]:
# Train Logistic Regression Model
y_pred_lr = lr_model.predict(Xf_test)
res_lr = evaluate_model("Logistic Regression (PCA)", y_test_bin, y_pred_lr)
joblib.dump(lr_model, os.path.join(OUTPUT_DIR, 'lr_model.pkl'))

# Train Random Forest Model
y_pred_rf = rf_model.predict(Xf_test)
res_rf = evaluate_model("Random Forest (PCA)", y_test_bin, y_pred_rf)
joblib.dump(rf_model, os.path.join(OUTPUT_DIR, 'rf_model.pkl'))

## CNN Model Building and Training (Binary Classification)

In [ ]:
# 1. Define CNN Architecture
def build_cnn(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))

    # --- Block 1: The "Spike Detector" (Low-level features) ---
    model.add(Conv1D(filters=32, kernel_size=10, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))

    # --- Block 2: The "Wave Detector" (Mid-level features) ---
    model.add(Conv1D(filters=64, kernel_size=10, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))

    # --- Block 3: The "Complex Shape Detector" (High-level features) ---
    # This layer sees the relationship between P, QRS, and T
    model.add(Conv1D(filters=128, kernel_size=10, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))

    # --- Block 4: Optional Deep Feature Extraction ---
    model.add(Conv1D(filters=256, kernel_size=10, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    # Global Average Pooling replaces Flatten (Reduces parameters, prevents overfitting)
    model.add(GlobalAveragePooling1D())

    # --- Classifier Head ---
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5)) # Standard dropout to force robustness
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(1, activation='sigmoid')) # Binary Output

    # Compile
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 3. Build
print(f"\n{'='*40}")
print("TRAINING CNN")
print(f"{'='*40}")

cnn = build_cnn((5000, 12))

callbacks_cnn = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001, verbose=1)
]

# 4. Train
history_cnn = cnn.fit(
    X_train, y_train_bin,
    validation_data=(X_test, y_test_bin),
    epochs=CNN_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks_cnn,
    verbose=1
)

#5. Save
cnn.save(os.path.join(OUTPUT_DIR, 'cnn_model_raw.h5'))

##  CNN Model Evaluation (Binary Classification)

In [ ]:
# Predict
print(f"\n{'='*50}")
print("CNN EVALUATION")
print(f"{'='*50}")

print("\nGenerating Predictions...")
y_pred_prob_cnn = cnn.predict(X_test)
y_pred_bin_cnn = (y_pred_prob_cnn > 0.5).astype(int).flatten()

# Evaluate
print("\nFinal Evaluation Results:")
res_cnn = evaluate_model("CNN (Raw)", y_test_bin, y_pred_bin_cnn)

## AG-ResVAE Model Building and Training (Binary Classification)

In [ ]:
# ==============================================================================
# 1. MODEL CONFIGURATION
# ==============================================================================
INPUT_SHAPE = (5000, 12)
NUM_CLASSES = 1
POOL_SIZE = 2
NUM_BLOCKS = 4
FILTERS_START = 64
LATENT_DIM = 128

LEARNING_RATE = 0.001
BATCH_SIZE = 16
EPOCHS = 50

WEIGHT_CLASSIFICATION = 1.0
WEIGHT_RECONSTRUCTION = 0.5
KL_LOSS_WEIGHT = 0.01

IDX_SEPTAL = [6, 7]
IDX_ANTERIOR = [8, 9]
IDX_LATERAL = [0, 4, 10, 11]
IDX_INFERIOR = [1, 2, 5]
IDX_POSTERIOR = [3]

# ==============================================================================
# 2. HELPER BLOCKS
# ==============================================================================

def residual_block(x, filters, kernel_size=7, stride=1):
    """Creates a Residual Block."""

    shortcut = x

    # Path A
    x = Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = SpatialDropout1D(0.3)(x)

    x = Conv1D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Path B
    if x.shape[-1] != shortcut.shape[-1] or stride != 1:
        shortcut = Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('swish')(x)
    return x

class Sampling(Layer):
    """VAE Sampling Layer."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class KLLossLayer(Layer):
    """Calculates KL Divergence Loss safely."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        self.add_loss(kl_loss * KL_LOSS_WEIGHT)
        return inputs

# ==============================================================================
# 3. ARCHITECTURE BUILDER
# ==============================================================================

def build_ag_resvae(input_shape=INPUT_SHAPE, latent_dim=LATENT_DIM):

    # --- BLOCK 1: INPUT & TRIAGE ---
    input_layer = Input(shape=input_shape, name="ecg_input")

    septal = Lambda(lambda x: tf.gather(x, IDX_SEPTAL, axis=2), name="Septal")(input_layer)
    anterior = Lambda(lambda x: tf.gather(x, IDX_ANTERIOR, axis=2), name="Anterior")(input_layer)
    lateral = Lambda(lambda x: tf.gather(x, IDX_LATERAL, axis=2), name="Lateral")(input_layer)
    inferior = Lambda(lambda x: tf.gather(x, IDX_INFERIOR, axis=2), name="Inferior")(input_layer)
    posterior = Lambda(lambda x: tf.gather(x, IDX_POSTERIOR, axis=2), name="Posterior")(input_layer)

    # --- BLOCK 2: ENCODER ---
    branches = []
    for region_name, branch_in in zip(["Sept", "Ant", "Lat", "Inf", "Post"],
                                      [septal, anterior, lateral, inferior, posterior]):

        # Initial Feature Extraction
        x = Conv1D(FILTERS_START, 7, strides=POOL_SIZE, padding='same', name=f"{region_name}_Conv1")(branch_in)
        x = BatchNormalization()(x)
        x = Activation('swish')(x)

        # Dynamic Residual Stacking
        current_filters = FILTERS_START * 2
        for i in range(NUM_BLOCKS):
            x = residual_block(x, current_filters, stride=POOL_SIZE)
            current_filters *= 2

        x = GlobalAveragePooling1D(name=f"{region_name}_GlobalPool")(x)
        branches.append(x)

    # --- BLOCK 3: BOTTLENECK (Latent Vector creation) ---
    merged = Concatenate(name="Anatomical_Fusion")(branches)

    z_mean = Dense(latent_dim, name="z_mean")(merged)
    z_log_var = Dense(latent_dim, name="z_log_var")(merged)
    _ = KLLossLayer()([z_mean, z_log_var])
    z = Sampling(name="z_sampling")([z_mean, z_log_var])

    # --- BLOCK 4: HEAD A (Diagnosis) ---
    c = Dense(256, activation='swish', kernel_initializer='he_normal')(z)
    c = BatchNormalization()(c)
    c = Dropout(0.4)(c)

    c = Dense(128, activation='swish', kernel_initializer='he_normal')(c)
    c = BatchNormalization()(c)
    c = Dropout(0.4)(c)

    classification_output = Dense(NUM_CLASSES, activation='sigmoid', name="classification")(c)

    # --- BLOCK 5: HEAD B (Reconstruction using Latent Vector) ---
    # Calculate Expected Bottleneck Size (Rounding UP)
    total_pools = 1 + NUM_BLOCKS
    downsample_factor = POOL_SIZE ** total_pools
    bottleneck_len = int(np.ceil(input_shape[0] / downsample_factor))

    # Expand Latent Vector (z)
    d = Dense(bottleneck_len * latent_dim, activation="swish")(z)
    d = Reshape((bottleneck_len, latent_dim))(d)

    # Upsample (Reverse Loop)
    filters_reverse = [2* FILTERS_START * (2**i) for i in range(NUM_BLOCKS)]
    filters_reverse = filters_reverse[::-1]

    for f in filters_reverse:
        d = Conv1DTranspose(f, 7, strides=POOL_SIZE, padding='same', activation='swish')(d)

    d = Conv1DTranspose(FILTERS_START, 7, strides=POOL_SIZE, padding='same', activation='swish')(d)

    # SAFETY CROP
    current_len = bottleneck_len * downsample_factor
    crop_amount = current_len - input_shape[0]

    if crop_amount > 0:
        d = Cropping1D(cropping=(0, crop_amount), name="Safety_Crop")(d)

    reconstruction_output = Conv1D(input_shape[1], 7, padding='same', activation='linear', name="reconstruction")(d)

    # --- BLOCK 6: COMPILATION ---
    model = Model(inputs=input_layer, outputs=[classification_output, reconstruction_output], name="AG_ResVAE")

    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss={
            "classification": "binary_crossentropy",
            "reconstruction": "mse"
        },
        loss_weights={
            "classification": WEIGHT_CLASSIFICATION,
            "reconstruction": WEIGHT_RECONSTRUCTION
        },
        metrics={"classification": "accuracy"}
    )

    return model

# ==============================================================================
# 4. EXECUTION PIPELINE
# ==============================================================================

print(f"{'='*50}")
print(f"AG-ResVAE TRAINING (OPTIMIZED)")
print(f"Input: {INPUT_SHAPE} | Latent Dim: {LATENT_DIM}")
print(f"Structure: {NUM_BLOCKS} Blocks | Start Filters: {FILTERS_START}")
print(f"{'='*50}")

# Build
model_ag_resvae = build_ag_resvae()

# Train
callbacks_ag_resvae = [
    ReduceLROnPlateau(
        monitor='val_classification_loss', factor=0.1, 
        patience=5, min_lr=1e-5, verbose=1, mode='min'
    ),
    EarlyStopping(
        monitor='val_classification_loss', patience=12, 
        restore_best_weights=True, verbose=1, mode='min'
    ),
    ModelCheckpoint(
        os.path.join(OUTPUT_DIR, "best_ag_resvae.keras"), 
        monitor='val_classification_accuracy', save_best_only=True, 
        verbose=1, mode='max'
    )
]

sample_weights_array = np.ones(shape=(len(y_train_bin),))
sample_weights_array[y_train_bin == 1] = 3.0
sample_weights_recon = np.ones(shape=(len(y_train_bin),))

print("\nStarting Training...")
history_ag = model_ag_resvae.fit(
    x=X_train,
    y={
        "classification": y_train_bin,
        "reconstruction": X_train
    },
    validation_data=(
        X_test,
        {"classification": y_test_bin, "reconstruction": X_test}
    ),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks_ag_resvae,
    sample_weight={
        "classification": sample_weights_array,
        "reconstruction": sample_weights_recon
    },
    verbose=1
)

## AG-ResVAE Model Evaluation (Binary Classification)

In [ ]:
# Predict
print(f"\n{'='*50}")
print("AG-ResVAE EVALUATION")
print(f"{'='*50}")

print("\nGenerating Predictions...")
all_preds = model_ag_resvae.predict(X_test)
y_pred_prob_ag = all_preds[0]
y_pred_bin_ag = (y_pred_prob_ag > 0.5).astype(int).flatten()

# Evaluate
print("\nFinal Evaluation Results:")
res_ag_resvae2 = evaluate_model("AG-ResVAE (RAW)", y_test_bin, y_pred_bin_ag)

## Final Result Of All Models

In [ ]:
# --- Standard Imports for Loading ---
import os
import joblib
from keras.models import load_model

print(f"{'='*50}")
print("FINAL PIPELINE: LOADING MODELS & PREDICTING")
print(f"{'='*50}\n")

# ==========================================
# 1. Load Machine Learning Models (Scikit-Learn)
# ==========================================

print("Loading ML Models...")
lr_loaded = joblib.load(os.path.join(OUTPUT_DIR, 'lr_model.pkl'))
rf_loaded = joblib.load(os.path.join(OUTPUT_DIR, 'rf_model.pkl'))

# ==========================================
# 2. Load Deep Learning Models (Keras)
# ==========================================

print("Loading DL Models...")
# Custom layer 'Sampling' and 'KLLossLayer' need to be passed for AG-ResVAE
cnn_loaded = load_model(os.path.join(OUTPUT_DIR, 'cnn_model_raw.h5'))
ag_resvae_loaded = load_model(
    os.path.join(OUTPUT_DIR, 'best_ag_resvae.keras'),
    custom_objects={'Sampling': Sampling, 'KLLossLayer': KLLossLayer}
)

print("All models loaded successfully!\n")

# ==========================================
# 3. Generate Predictions on Test Data
# ==========================================

# ML Predictions (using PCA features)
y_pred_lr = lr_loaded.predict(Xf_test)
y_pred_rf = rf_loaded.predict(Xf_test)

# DL Predictions (using Raw signals)
y_pred_prob_cnn = cnn_loaded.predict(X_test, verbose=0)
y_pred_cnn = (y_pred_prob_cnn > 0.5).astype(int).flatten()

ag_resvae_preds = ag_resvae_loaded.predict(X_test, verbose=0)
y_pred_prob_ag = ag_resvae_preds[0] # [0] is classification head
y_pred_ag = (y_pred_prob_ag > 0.5).astype(int).flatten()

# ==========================================
# 4. Final Evaluation
# ==========================================

print("\n--- Final Test Set Evaluations ---")
final_lr = evaluate_model("Loaded Logistic Regression", y_test_bin, y_pred_lr)
final_rf = evaluate_model("Loaded Random Forest", y_test_bin, y_pred_rf)
final_cnn = evaluate_model("Loaded CNN", y_test_bin, y_pred_cnn)
final_ag = evaluate_model("Loaded AG-ResVAE", y_test_bin, y_pred_ag)

# Consolidate results into a final DataFrame
final_results_df = pd.DataFrame([final_lr, final_rf, final_cnn, final_ag])
display(final_results_df.style.format({
    'accuracy': '{:.2%}',
    'precision': '{:.2%}',
    'recall': '{:.2%}',
    'f1 score': '{:.2%}'
}))

# 📌 Conclusion

We trained and compared **three models** for MI detection:

### ✔ Logistic Regression (very fast, uses simple features)  
### ✔ Random Forest (better than LR in many cases)  
### ✔ 1D CNN (uses raw ECG, learns automatically)

Evaluation metrics included:
- Accuracy  
- Precision  
- Recall  
- F1  
- Confusion Matrix  